In [ ]:
import pandas as pd
import numpy as np
import seaborn as sns 
import matplotlib.pyplot as plt

%matplotlib inline
import warnings
warnings.filterwarnings("ignore")
import scipy.stats as stats
import statsmodels.api as sm
from scipy.stats import genextreme

# preprocessing

In [ ]:
df_given= pd.read_csv("Analysis_.csv")
df_generated= pd.read_csv("Output_.csv")

In [ ]:
df_given.dropna(inplace=True)
df_generated.dropna(inplace=True)

In [ ]:
df_given= df_given.iloc[:len(df_generated),:]
df_generated= df_generated

In [ ]:
df_given

In [ ]:
df_generated

# IDF curve

In [ ]:
def idf(df):
    duration = [1, 3, 6, 12, 24]
    table = np.zeros((len(df['YEAR'].unique()), len(duration)), dtype=float)
    
    # Loop through each year
    for i, year in enumerate(df['YEAR'].unique()):
        # Extract rainfall values for the current year
        rainfall_val = df[df['YEAR'] == year].values[:, 5:29]
        
        # Loop through each duration
        for j, dur in enumerate(duration):
            value = []
            
            # Calculate the sum of rainfall for the specified duration
            for k in range(len(rainfall_val.flatten()) - dur + 1):
                value.append(np.sum(rainfall_val.flatten()[k:k + dur])) 
            
            # Store the maximum value in the table
            table[i, j] = max(value)
    
    mean_val= np.mean(table,axis=0)
    std_dev = np.std(table,axis=0)
    
    kt= [-0.164,0.719,1.305,2.592,3.137]
    duration= [1,3,6,12,24]
    
    # the IDF table
    
    idf= np.zeros((len(kt),len(mean_val)),dtype= float)
    
    for i in range(len(kt)):
        for j in range(len(mean_val)):
            
            idf[i,j]= (mean_val[j]+(kt[i]*std_dev[j]))/duration[j]
            
    return idf

In [ ]:
def plot_idf(df):
    
    return_period= ["2 year","5 year","10 year","20 year","50 year","100 year"]
    duration     = [1,3,6,12,24]
    
    for i,(row,label) in enumerate(zip(df,return_period)):
        plt.plot(duration,row,label=label)
        
    plt.xlabel("duration")
    plt.ylabel("intensity")
    plt.title("IDf curve for the generated data")
    plt.grid(True)
    plt.legend()
    plt.show()

In [ ]:
def compare(idf_given,idf_generated):
    rms_error = np.sqrt(((idf_given - idf_generated) ** 2).mean(axis=1))
    percentage_error = (np.abs(idf_given - idf_generated) /idf_given) * 100
    mean_percentage_error = percentage_error.mean(axis=1)
    return_period= ["2 year","5 year","10 year","20 year","50 year"]
    rms_error_value = np.sqrt(np.mean(np.square(idf_given - idf_generated)))
    print("Obtained RMSE:",rms_error_value)
    print("Obtained % error:",mean_percentage_error)
    
    plt.figure(figsize=(12, 6))
    
    plt.subplot(1, 2, 1)
    sns.barplot(x=return_period, y=mean_percentage_error, palette="viridis")
    plt.title('% Error of IDF curves')
    plt.ylabel('Value')
    
    plt.subplot(1, 2, 2)
    sns.barplot(x=return_period, y=rms_error, palette="viridis")
    plt.title('RMS Error of IDF curves')
    plt.ylabel('Value')  
    

In [ ]:
given_idf= idf(df_given)
plot_idf(given_idf)

In [ ]:
generated_idf= idf(df_generated)
plot_idf(generated_idf)

In [ ]:
compare(given_idf,generated_idf)

# preprocessing after idf

In [ ]:
df_given.drop(columns=['INDEX', 'YEAR', 'MN', 'DT', 'DAY','TOTRF','TOT_Cal'],inplace=True)
df_generated.drop(columns=['INDEX', 'YEAR', 'MN', 'DT', 'DAY','TOTRF','TOT_Cal'],inplace=True)

# percentage 0 values

In [ ]:
#Percentage zero values

def compare_dry_spell(df,df_generated):
    compare= [0,0]
    compare[0]= (df==0).sum().sum()
    compare[1]= (df_generated==0).sum().sum()
    
    total= [0,0]
    total[0]= df.size
    total[1]= df_generated.size
    
    print('Percent zero values in original dataset:',100*(compare[0]/total[0]))
    print('Percent zero values in generated dataset:',100*(compare[1]/total[1])) 
    
    labels = ['df', 'df_generated']
    plt.bar(labels, compare)
    plt.xlabel('Dataset')
    plt.ylabel('Count of Zeros')
    plt.title('Comparison of Zeros in Datasets')
    plt.show()

In [ ]:
compare_dry_spell(df_given,df_generated)

# given total rainfall vs calculated total rainfall

In [ ]:
a= np.sum(df_given.values[:,:],axis=1)
b= np.sum(df_generated.values[:,:],axis=1)
rms_error_value = np.sqrt(np.mean(np.square(a - b)))
print('RMS error in given total rainfall vs generated rainfall:',rms_error_value)

plt.plot(a,label='given_data')
plt.plot(b,label='generated_data')
plt.xlabel('Days')
plt.ylabel('Total Rainfall')
plt.legend()
plt.show()

# quantile plots, means & std deviations

In [ ]:
def plot(df, df_generated):
    given_values = df.values.flatten()
    generated_values = df_generated.values.flatten()
    
    dry_period_given = []
    dry_period_generated = []
    event_given = []
    event_generated = []
    vol_given = []
    vol_generated = []
    
    flag = 0
    count2 = 0
    for i in range(len(given_values) - 1):
        if flag == 1:
            count2 += 1
        if given_values[i] != 0 and given_values[i+1] == 0:
            flag = 1
        if given_values[i] == 0 and given_values[i+1] != 0:
            dry_period_given.append(count2)
            count2 = 0
            flag = 0
            
    flag = 0
    count2 = 0
    for i in range(len(generated_values) - 1):
        if flag == 1:
            count2 += 1
        if generated_values[i] != 0 and generated_values[i+1] == 0:
            flag = 1
        if generated_values[i] == 0 and generated_values[i+1] != 0:
            dry_period_generated.append(count2)
            count2 = 0
            flag = 0  
    
    flag = 0
    count = 0
    count1 = 0
    for i in range(len(given_values) - 1):
        if flag == 1:
            count += 1
            count1 += given_values[i]
        if given_values[i] == 0 and given_values[i+1] != 0:
            flag = 1
        if given_values[i] != 0 and given_values[i+1] == 0:
            event_given.append(count)
            vol_given.append(count1)
            count1 = 0
            count = 0
            flag = 0

    flag = 0
    count = 0
    count1 = 0
    for i in range(len(generated_values) - 1):
        if flag == 1:
            count += 1
            count1 += generated_values[i]
        if generated_values[i] == 0 and generated_values[i+1] != 0:
            flag = 1
        if generated_values[i] != 0 and generated_values[i+1] == 0:
            event_generated.append(count)
            vol_generated.append(count1)
            count1 = 0
            count = 0
            flag = 0
    
    event_given = np.array(event_given)
    event_generated = np.array(event_generated)
    vol_given = np.array(vol_given)
    vol_generated = np.array(vol_generated)
    dry_period_given = np.array(dry_period_given)
    dry_period_generated = np.array(dry_period_generated)
    
    # Mean of event volume:    
    mean_given = np.mean(vol_given)
    mean_generated = np.mean(vol_generated)
    print('________________________________________________________________________________________________________________')
    print('Mean of event volume in original dataset:', mean_given)
    print('Mean of event volume in generated dataset:', mean_generated)
    print('________________________________________________________________________________________________________________')
    
    # Std dev of event volume:
    given_sd_rain = np.std(vol_given)
    generated_sd_rain = np.std(vol_generated)
    print('________________________________________________________________________________________________________________')
    print('Std Dev. of event volume in original dataset:', given_sd_rain)
    print('Std Dev. of event volume in generated dataset:', generated_sd_rain)
    print('________________________________________________________________________________________________________________')
    
    # Mean of event duration:    
    mean__event_given = np.mean(event_given)
    mean__event_generated = np.mean(event_generated)
    print('________________________________________________________________________________________________________________')
    print('Mean of event duration in original dataset:', mean__event_given)
    print('Mean of event duration in generated dataset:', mean__event_generated)
    print('________________________________________________________________________________________________________________')
    
    # Std dev of event duration:
    given_sd_event = np.std(event_given)
    generated_sd_event = np.std(event_generated)
    print('________________________________________________________________________________________________________________')
    print('Std Dev. of event duration in original dataset:', given_sd_event)
    print('Std Dev. of event duration in generated dataset:', generated_sd_event)
    print('________________________________________________________________________________________________________________')
    
    # Bar plots for mean and standard deviation
    labels = ['Original', 'Generated']
    
    mean_vols = [mean_given, mean_generated]
    std_vols = [given_sd_rain, generated_sd_rain]
    
    mean_events = [mean__event_given, mean__event_generated]
    std_events = [given_sd_event, generated_sd_event]
    
    plt.figure(figsize=(12, 6))
    plt.subplot(1, 2, 1)
    sns.barplot(x=labels, y=mean_vols, palette="viridis")
    plt.title('Mean Event Volume')
    plt.ylabel('Volume')
    
    plt.subplot(1, 2, 2)
    sns.barplot(x=labels, y=std_vols, palette="viridis")
    plt.title('Standard Deviation of Event Volume')
    plt.ylabel('Volume')
    
    plt.tight_layout()
    plt.show()
    
    plt.figure(figsize=(12, 6))
    plt.subplot(1, 2, 1)
    sns.barplot(x=labels, y=mean_events, palette="coolwarm")
    plt.title('Mean Event Duration')
    plt.ylabel('Duration')
    
    plt.subplot(1, 2, 2)
    sns.barplot(x=labels, y=std_events, palette="coolwarm")
    plt.title('Standard Deviation of Event Duration')
    plt.ylabel('Duration')
    
    plt.tight_layout()
    plt.show()
    
    # QQ plots
    fig, axes = plt.subplots(1, 3, figsize=(18, 6))
    
    sm.qqplot_2samples(vol_given, vol_generated, line='45', ax=axes[0])
    axes[0].set_title('QQ Plot of Event Volume')
    axes[0].set_xlabel('Given Event Volume')
    axes[0].set_ylabel('Generated Event Volume')
    
    sm.qqplot_2samples(event_given, event_generated, line='45', ax=axes[1])
    axes[1].set_title('QQ Plot of Event Duration')
    axes[1].set_xlabel('Given Event Duration')
    axes[1].set_ylabel('Generated Event Duration')
    
    sm.qqplot_2samples(dry_period_given, dry_period_generated, line='45', ax=axes[2])
    axes[2].set_title('QQ Plot of Dry Period Lengths')
    axes[2].set_xlabel('Given Dry Period Lengths')
    axes[2].set_ylabel('Generated Dry Period Lengths')
    
    plt.tight_layout()
    plt.show()

# Sample usage (you need to replace df and df_generated with your actual dataframes)
# plot(df, df_generated)


In [ ]:
plot(df_given, df_generated)